Stage 1: Data Preprocessing and Tokenization (Optimized)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer
from datasets import Dataset
import os
import shutil

# Function to check available disk space
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Load dataset
data_path = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
df = pd.read_csv(data_path)

# Clean dataset
df = df.drop_duplicates(subset=["comment_text"])
df = df[["comment_text", "toxic"]].dropna()

# Balance classes
toxic_df = df[df["toxic"] == 1]
non_toxic_df = df[df["toxic"] == 0].sample(n=len(toxic_df), random_state=42)
balanced_df = pd.concat([toxic_df, non_toxic_df]).sample(frac=1, random_state=42)

# Split dataset
train_df, temp_df = train_test_split(balanced_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Rename for Trainer compatibility
train_df = train_df.rename(columns={"toxic": "labels"})
val_df = val_df.rename(columns={"toxic": "labels"})
test_df = test_df.rename(columns={"toxic": "labels"})

# Convert to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["comment_text"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Initialize tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Create output directory
os.makedirs("/kaggle/working/preprocessed", exist_ok=True)

# Tokenize and save as compressed Parquet
models = ["bert", "roberta", "distilbert"]
tokenizers = [bert_tokenizer, roberta_tokenizer, distilbert_tokenizer]

for model_name, tokenizer in zip(models, tokenizers):
    check_disk_space()
    train_ds = tokenize_dataset(train_dataset, tokenizer)
    val_ds = tokenize_dataset(val_dataset, tokenizer)
    test_ds = tokenize_dataset(test_dataset, tokenizer)
    
    # Save as compressed Parquet
    train_ds.to_parquet(f"/kaggle/working/preprocessed/train_{model_name}.parquet", compression="gzip")
    val_ds.to_parquet(f"/kaggle/working/preprocessed/val_{model_name}.parquet", compression="gzip")
    test_ds.to_parquet(f"/kaggle/working/preprocessed/test_{model_name}.parquet", compression="gzip")

print("Preprocessing completed. Compressed Parquet datasets saved to /kaggle/working/preprocessed/")
check_disk_space()

Free disk space: 19.50 GB


Map:   0%|          | 0/21411 [00:00<?, ? examples/s]

Map:   0%|          | 0/4588 [00:00<?, ? examples/s]

Map:   0%|          | 0/4589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Preprocessing completed. Compressed Parquet datasets saved to /kaggle/working/preprocessed/
Free disk space: 19.48 GB


19.483043670654297

Stage 2: Model Fine-Tuning (Optimized)

In [2]:
from transformers import (BertForSequenceClassification, RobertaForSequenceClassification, 
                          DistilBertForSequenceClassification, Trainer, TrainingArguments)
from datasets import Dataset, load_dataset
import os
import shutil

# Disk space check function
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Load preprocessed Parquet datasets
models = ["bert", "roberta", "distilbert"]
train_datasets = {}
val_datasets = {}
for model in models:
    train_datasets[model] = load_dataset("parquet", data_files=f"/kaggle/working/preprocessed/train_{model}.parquet")["train"]
    val_datasets[model] = load_dataset("parquet", data_files=f"/kaggle/working/preprocessed/val_{model}.parquet")["train"]

# Initialize models
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
distilbert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/models",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    save_total_limit=1,
)

# Fine-tune function
def fine_tune_model(model, train_dataset, val_dataset, model_name):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )
    trainer.train()
    trainer.save_model(f"/kaggle/working/models/{model_name}")
    return trainer

# Fine-tune and save
os.makedirs("/kaggle/working/models", exist_ok=True)
check_disk_space()

bert_trainer = fine_tune_model(bert_model, train_datasets["bert"], val_datasets["bert"], "bert_finetuned")
roberta_trainer = fine_tune_model(roberta_model, train_datasets["roberta"], val_datasets["roberta"], "roberta_finetuned")
distilbert_trainer = fine_tune_model(distilbert_model, train_datasets["distilbert"], val_datasets["distilbert"], "distilbert_finetuned")

# Save tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_tokenizer.save_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer.save_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer.save_pretrained("/kaggle/working/models/distilbert_finetuned")

# Clean up preprocessed datasets
shutil.rmtree("/kaggle/working/preprocessed", ignore_errors=True)
print("Fine-tuning completed. Models saved to /kaggle/working/models/. Preprocessed data cleaned up.")
check_disk_space()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Free disk space: 19.48 GB


Fine-tuning completed. Models saved to /kaggle/working/models/. Preprocessed data cleaned up.
Free disk space: 17.63 GB


17.630462646484375

Stage 3: Model Evaluation (Optimized)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer
from datasets import Dataset
import os

# Load dataset
data_path = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
df = pd.read_csv(data_path)

# Clean dataset
df = df.drop_duplicates(subset=["comment_text"])
df = df[["comment_text", "toxic"]].dropna()

# Balance classes
toxic_df = df[df["toxic"] == 1]
non_toxic_df = df[df["toxic"] == 0].sample(n=len(toxic_df), random_state=42)
balanced_df = pd.concat([toxic_df, non_toxic_df]).sample(frac=1, random_state=42)

# Split dataset (same as Stage 1)
train_df, temp_df = train_test_split(balanced_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Rename for Trainer compatibility
test_df = test_df.rename(columns={"toxic": "labels"})

# Convert to Dataset
test_dataset = Dataset.from_pandas(test_df)

# Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["comment_text"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Initialize tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Create output directory
os.makedirs("/kaggle/working/preprocessed", exist_ok=True)

# Tokenize and save only test datasets
models = ["bert", "roberta", "distilbert"]
tokenizers = [bert_tokenizer, roberta_tokenizer, distilbert_tokenizer]
for model_name, tokenizer in zip(models, tokenizers):
    test_ds = tokenize_dataset(test_dataset, tokenizer)
    test_ds.to_parquet(f"/kaggle/working/preprocessed/test_{model_name}.parquet", compression="gzip")

print("Test datasets regenerated and saved to /kaggle/working/preprocessed/")

Map:   0%|          | 0/4589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Map:   0%|          | 0/4589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Map:   0%|          | 0/4589 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Test datasets regenerated and saved to /kaggle/working/preprocessed/


In [5]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import pandas as pd
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import os
import shutil

# Disk space check function
def check_disk_space():
    stat = os.statvfs("/kaggle/working")
    free_space = stat.f_frsize * stat.f_bavail / (1024 ** 3)  # GB
    print(f"Free disk space: {free_space:.2f} GB")
    return free_space

# Load test datasets from Parquet
models = ["bert", "roberta", "distilbert"]
test_datasets = {model: load_dataset("parquet", data_files=f"/kaggle/working/preprocessed/test_{model}.parquet")["train"] for model in models}

# Load fine-tuned models
bert_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Training arguments for evaluation
training_args = TrainingArguments(
    output_dir="/kaggle/working/models",
    eval_strategy="epoch",
    per_device_eval_batch_size=8,
    report_to="none"
)

# Metrics computation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = pred.predictions[:, 1]
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
        "auc_roc": roc_auc_score(labels, probs),
        "confusion_matrix": confusion_matrix(labels, preds).ravel().tolist()
    }

# Initialize trainers
trainers = {
    "bert": Trainer(model=bert_model, args=training_args, eval_dataset=test_datasets["bert"], compute_metrics=compute_metrics),
    "roberta": Trainer(model=roberta_model, args=training_args, eval_dataset=test_datasets["roberta"], compute_metrics=compute_metrics),
    "distilbert": Trainer(model=distilbert_model, args=training_args, eval_dataset=test_datasets["distilbert"], compute_metrics=compute_metrics)
}

# Evaluate models
results = {"Model": [], "Accuracy": [], "F1": [], "Precision": [], "Recall": [], "AUC-ROC": [], "Confusion Matrix": []}
for model_name, trainer in trainers.items():
    eval_result = trainer.evaluate()
    results["Model"].append(model_name.capitalize())
    results["Accuracy"].append(eval_result["eval_accuracy"])
    results["F1"].append(eval_result["eval_f1"])
    results["Precision"].append(eval_result["eval_precision"])
    results["Recall"].append(eval_result["eval_recall"])
    results["AUC-ROC"].append(eval_result["eval_auc_roc"])
    results["Confusion Matrix"].append(eval_result["eval_confusion_matrix"])

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("/kaggle/working/model_performance.csv", index=False)

# Clean up test datasets if they exist
if os.path.exists("/kaggle/working/preprocessed"):
    shutil.rmtree("/kaggle/working/preprocessed", ignore_errors=True)

print("Evaluation completed. Results saved to /kaggle/working/model_performance.csv")
print(results_df)
check_disk_space()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Evaluation completed. Results saved to /kaggle/working/model_performance.csv
        Model  Accuracy        F1  Precision    Recall   AUC-ROC  \
0        Bert  0.925256  0.927896   0.901184  0.956239  0.981047   
1     Roberta  0.933537  0.933997   0.932987  0.935009  0.978457   
2  Distilbert  0.929614  0.931057   0.917543  0.944974  0.979075   

         Confusion Matrix  
0  [2039, 242, 101, 2207]  
1  [2126, 155, 150, 2158]  
2  [2085, 196, 127, 2181]  
Free disk space: 17.63 GB


17.63045883178711

Before Disk compression

Step 1: Preprocess Jigsaw Dataset (Clean, Balance, Split, Tokenize)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer
from datasets import Dataset

# Load dataset from Kaggle input
data_path = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
df = pd.read_csv(data_path)

# Clean dataset
df = df.drop_duplicates(subset=["comment_text"])
df = df[["comment_text", "toxic"]].dropna()

# Balance classes (optional)
toxic_df = df[df["toxic"] == 1]
non_toxic_df = df[df["toxic"] == 0].sample(n=len(toxic_df), random_state=42)
balanced_df = pd.concat([toxic_df, non_toxic_df]).sample(frac=1, random_state=42)

# Split: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(balanced_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Rename 'toxic' to 'labels' for Trainer compatibility
train_df = train_df.rename(columns={"toxic": "labels"})
val_df = val_df.rename(columns={"toxic": "labels"})
test_df = test_df.rename(columns={"toxic": "labels"})

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["comment_text"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Initialize tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize for each model
train_bert = tokenize_dataset(train_dataset, bert_tokenizer)
val_bert = tokenize_dataset(val_dataset, bert_tokenizer)
test_bert = tokenize_dataset(test_dataset, bert_tokenizer)

train_roberta = tokenize_dataset(train_dataset, roberta_tokenizer)
val_roberta = tokenize_dataset(val_dataset, roberta_tokenizer)
test_roberta = tokenize_dataset(test_dataset, roberta_tokenizer)

train_distilbert = tokenize_dataset(train_dataset, distilbert_tokenizer)
val_distilbert = tokenize_dataset(val_dataset, distilbert_tokenizer)
test_distilbert = tokenize_dataset(test_dataset, distilbert_tokenizer)

# Save to Kaggle working directory
train_bert.save_to_disk("/kaggle/working/preprocessed/train_bert")
val_bert.save_to_disk("/kaggle/working/preprocessed/val_bert")
test_bert.save_to_disk("/kaggle/working/preprocessed/test_bert")
train_roberta.save_to_disk("/kaggle/working/preprocessed/train_roberta")
val_roberta.save_to_disk("/kaggle/working/preprocessed/val_roberta")
test_roberta.save_to_disk("/kaggle/working/preprocessed/test_roberta")
train_distilbert.save_to_disk("/kaggle/working/preprocessed/train_distilbert")
val_distilbert.save_to_disk("/kaggle/working/preprocessed/val_distilbert")
test_distilbert.save_to_disk("/kaggle/working/preprocessed/test_distilbert")

Step 2: Fine-Tune BERT, DistilBERT, RoBERTa with Validation Set

In [ ]:
from transformers import (BertForSequenceClassification, RobertaForSequenceClassification, 
                          DistilBertForSequenceClassification, Trainer, TrainingArguments)
import torch

# Load preprocessed datasets
train_bert = Dataset.load_from_disk("/kaggle/working/preprocessed/train_bert")
val_bert = Dataset.load_from_disk("/kaggle/working/preprocessed/val_bert")
train_roberta = Dataset.load_from_disk("/kaggle/working/preprocessed/train_roberta")
val_roberta = Dataset.load_from_disk("/kaggle/working/preprocessed/val_roberta")
train_distilbert = Dataset.load_from_disk("/kaggle/working/preprocessed/train_distilbert")
val_distilbert = Dataset.load_from_disk("/kaggle/working/preprocessed/val_distilbert")

# Load models
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
distilbert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/models",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"
)

# Fine-tune function
def fine_tune_model(model, train_dataset, val_dataset, model_name):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )
    trainer.train()
    trainer.save_model(f"/kaggle/working/models/{model_name}")
    return trainer

# Fine-tune each model
bert_trainer = fine_tune_model(bert_model, train_bert, val_bert, "bert_finetuned")
roberta_trainer = fine_tune_model(roberta_model, train_roberta, val_roberta, "roberta_finetuned")
distilbert_trainer = fine_tune_model(distilbert_model, train_distilbert, val_distilbert, "distilbert_finetuned")

# Save tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_tokenizer.save_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer.save_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer.save_pretrained("/kaggle/working/models/distilbert_finetuned")

Step 3: Evaluate on Test Set and Document Results

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import pandas as pd
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load test datasets
test_bert = Dataset.load_from_disk("/kaggle/working/preprocessed/test_bert")
test_roberta = Dataset.load_from_disk("/kaggle/working/preprocessed/test_roberta")
test_distilbert = Dataset.load_from_disk("/kaggle/working/preprocessed/test_distilbert")

# Load fine-tuned models
bert_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Define training args for evaluation (no training, just eval)
training_args = TrainingArguments(
    output_dir="/kaggle/working/models",
    eval_strategy="epoch",  # Still works since we provide eval_dataset
    per_device_eval_batch_size=8,
    report_to="none"
)

# Compute metrics including AUC-ROC and confusion matrix
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Predicted classes
    probs = pred.predictions[:, 1]  # Probability scores for the positive class (assuming binary classification)

    # Standard classification metrics
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    
    # AUC-ROC (requires probability scores)
    auc_roc = roc_auc_score(labels, probs)
    
    # Confusion matrix (returns a 2x2 array for binary classification)
    conf_matrix = confusion_matrix(labels, preds).ravel().tolist()  # Flatten to [TN, FP, FN, TP]

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "auc_roc": auc_roc,
        "confusion_matrix": conf_matrix  # Returns list like [TN, FP, FN, TP]
    }

# Evaluate each model with its test dataset
bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    eval_dataset=test_bert,
    compute_metrics=compute_metrics
)
roberta_trainer = Trainer(
    model=roberta_model,
    args=training_args,
    eval_dataset=test_roberta,
    compute_metrics=compute_metrics
)
distilbert_trainer = Trainer(
    model=distilbert_model,
    args=training_args,
    eval_dataset=test_distilbert,
    compute_metrics=compute_metrics
)

# Run evaluation
bert_eval = bert_trainer.evaluate()
roberta_eval = roberta_trainer.evaluate()
distilbert_eval = distilbert_trainer.evaluate()

# Document results
results = {
    "Model": ["BERT", "RoBERTa", "DistilBERT"],
    "Accuracy": [bert_eval["eval_accuracy"], roberta_eval["eval_accuracy"], distilbert_eval["eval_accuracy"]],
    "F1": [bert_eval["eval_f1"], roberta_eval["eval_f1"], distilbert_eval["eval_f1"]],
    "Precision": [bert_eval["eval_precision"], roberta_eval["eval_precision"], distilbert_eval["eval_precision"]],
    "Recall": [bert_eval["eval_recall"], roberta_eval["eval_recall"], distilbert_eval["eval_recall"]],
    "AUC-ROC": [bert_eval["eval_auc_roc"], roberta_eval["eval_auc_roc"], distilbert_eval["eval_auc_roc"]],
    "Confusion Matrix": [bert_eval["eval_confusion_matrix"], roberta_eval["eval_confusion_matrix"], distilbert_eval["eval_confusion_matrix"]]
}
results_df = pd.DataFrame(results)
results_df.to_csv("/kaggle/working/model_performance.csv", index=False)
print(results_df)

Step 4: Test on Curated Toxic/Non-Toxic Examples

In [6]:
import torch
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer

# Load tokenizers
bert_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Move models to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
roberta_model.to(device)
distilbert_model.to(device)

# Curated examples
examples = [
    {"text": "You’re a complete idiot!", "label": 1},
    {"text": "Nice work, well done!", "label": 0},
    {"text": "Die, you worthless scum.", "label": 1},
    {"text": "This is a great discussion.", "label": 0},
]

# Prediction function (ensure inputs are on the same device as model)
def predict_toxicity(model, tokenizer, text, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient computation for inference
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)
    pred = probs.argmax(-1).item()
    return pred, probs[0][1].item()

# Test each model
for example in examples:
    text, true_label = example["text"], example["label"]
    print(f"\nText: {text}")
    for model_name, model, tokenizer in [
        ("BERT", bert_model, bert_tokenizer),
        ("RoBERTa", roberta_model, roberta_tokenizer),
        ("DistilBERT", distilbert_model, distilbert_tokenizer),
    ]:
        pred, prob = predict_toxicity(model, tokenizer, text, device)
        print(f"{model_name}: Predicted={pred}, Toxic Prob={prob:.4f}, True={true_label}")


Text: You’re a complete idiot!
BERT: Predicted=1, Toxic Prob=0.9993, True=1
RoBERTa: Predicted=1, Toxic Prob=0.9909, True=1
DistilBERT: Predicted=1, Toxic Prob=0.9989, True=1

Text: Nice work, well done!
BERT: Predicted=0, Toxic Prob=0.0079, True=0
RoBERTa: Predicted=0, Toxic Prob=0.0096, True=0
DistilBERT: Predicted=0, Toxic Prob=0.0078, True=0

Text: Die, you worthless scum.
BERT: Predicted=1, Toxic Prob=0.9994, True=1
RoBERTa: Predicted=1, Toxic Prob=0.9909, True=1
DistilBERT: Predicted=1, Toxic Prob=0.9987, True=1

Text: This is a great discussion.
BERT: Predicted=0, Toxic Prob=0.0055, True=0
RoBERTa: Predicted=0, Toxic Prob=0.0093, True=0
DistilBERT: Predicted=0, Toxic Prob=0.0079, True=0


Step 5: Prepare Models for Web App (Test Inference, Optimize)

In [7]:
from transformers import pipeline
import time

# Use DistilBERT for inference
classifier = pipeline("text-classification", model="/kaggle/working/models/distilbert_finetuned", 
                      tokenizer=distilbert_tokenizer)

# Test inference
start_time = time.time()
result = classifier("You’re a disgusting human being!")
end_time = time.time()
print(f"Prediction: {result}, Latency: {end_time - start_time:.4f}s")

# Additional test for confirmation
start_time = time.time()
result = classifier("Great job, keep it up!")
end_time = time.time()
print(f"Prediction: {result}, Latency: {end_time - start_time:.4f}s")

Device set to use cuda:0


Prediction: [{'label': 'LABEL_1', 'score': 0.9984644651412964}], Latency: 0.0056s
Prediction: [{'label': 'LABEL_0', 'score': 0.9634913802146912}], Latency: 0.0052s


Testing with Youtube Comments

In [8]:
#Testing on youtube comments datasets

import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset

# Step 1: Load CSV file from Kaggle environment
csv_file_path = "/kaggle/input/youtube-toxicity-data/youtoxic_english_1000.csv"  # Replace with your actual CSV file path
comments_df = pd.read_csv(csv_file_path)

# Ensure the column name matches your CSV file (adjust if different)
if "Text" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
else:
    print(f"Loaded CSV from {csv_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["Text"]])

# Step 3: Tokenizers for the three fine-tuned models
bert_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_bert = tokenize_dataset(comments_dataset, bert_tokenizer)
comments_roberta = tokenize_dataset(comments_dataset, roberta_tokenizer)
comments_distilbert = tokenize_dataset(comments_dataset, distilbert_tokenizer)

# Step 5: Load the models
bert_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
bert_model.to(device)
roberta_model.to(device)
distilbert_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["Text"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=64)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with BERT...")
bert_preds, bert_probs = predict_toxicity(bert_model, bert_tokenizer, comments_bert, device)

print("Predicting with RoBERTa...")
roberta_preds, roberta_probs = predict_toxicity(roberta_model, roberta_tokenizer, comments_roberta, device)

print("Predicting with DistilBERT...")
distilbert_preds, distilbert_probs = predict_toxicity(distilbert_model, distilbert_tokenizer, comments_distilbert, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["Text"],
    "BERT_Pred": bert_preds,
    "BERT_Toxic_Prob": bert_probs,
    "RoBERTa_Pred": roberta_preds,
    "RoBERTa_Toxic_Prob": roberta_probs,
    "DistilBERT_Pred": distilbert_preds,
    "DistilBERT_Toxic_Prob": distilbert_probs
})

results_csv_path = "/kaggle/working/combined_English_model_predictions_YTD.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection


Loaded CSV from /kaggle/input/youtube-toxicity-data/youtoxic_english_1000.csv with 1000 comments.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Predicting with BERT...
Predicting with RoBERTa...
Predicting with DistilBERT...
Predictions saved to /kaggle/working/combined_English_model_predictions_YTD.csv
                                             Comment  BERT_Pred  \
0  If only people would just take a step back and...          1   
1  Law enforcement is not trained to shoot to app...          1   
2  \nDont you reckon them 'black lives matter' ba...          1   
3  There are a very large number of people who do...          0   
4  The Arab dude is absolutely right, he should h...          1   
5  here people his facebook is https://www.facebo...          1   
6  Check out this you tube post. "Black man goes ...          1   
7  I would LOVE to see this pussy go to Staten Is...          1   
8                        I agree with the protestor.          0   
9   mike browns father was made to say that boooshit          1   

   BERT_Toxic_Prob  RoBERTa_Pred  RoBERTa_Toxic_Prob  DistilBERT_Pred  \
0         0.611693          

In [9]:
#Testing on youtube comments datasets

import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset

# Step 1: Load Excel file and convert to CSV
excel_file_path = "/kaggle/input/random-multi-youtube-comments-scrapped/Youtube comments scrapper_Testing File.xlsx"  # Replace with your actual file path
comments_df = pd.read_excel(excel_file_path)

# Ensure the column name matches your CSV file (adjust if different)
if "comment" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
else:
    print(f"Loaded CSV from {csv_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["comment"]])

# Step 3: Tokenizers for the three fine-tuned models
bert_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["comment"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_bert = tokenize_dataset(comments_dataset, bert_tokenizer)
comments_roberta = tokenize_dataset(comments_dataset, roberta_tokenizer)
comments_distilbert = tokenize_dataset(comments_dataset, distilbert_tokenizer)

# Step 5: Load the models
bert_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
bert_model.to(device)
roberta_model.to(device)
distilbert_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["comment"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=64)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with BERT...")
bert_preds, bert_probs = predict_toxicity(bert_model, bert_tokenizer, comments_bert, device)

print("Predicting with RoBERTa...")
roberta_preds, roberta_probs = predict_toxicity(roberta_model, roberta_tokenizer, comments_roberta, device)

print("Predicting with DistilBERT...")
distilbert_preds, distilbert_probs = predict_toxicity(distilbert_model, distilbert_tokenizer, comments_distilbert, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["comment"],
    "BERT_Pred": bert_preds,
    "BERT_Toxic_Prob": bert_probs,
    "RoBERTa_Pred": roberta_preds,
    "RoBERTa_Toxic_Prob": roberta_probs,
    "DistilBERT_Pred": distilbert_preds,
    "DistilBERT_Toxic_Prob": distilbert_probs
})

results_csv_path = "/kaggle/working/combined_English_model_predictions_RMYC.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection


Loaded CSV from /kaggle/input/youtube-toxicity-data/youtoxic_english_1000.csv with 563 comments.


Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Predicting with BERT...
Predicting with RoBERTa...
Predicting with DistilBERT...
Predictions saved to /kaggle/working/combined_English_model_predictions_RMYC.csv
                                             Comment  BERT_Pred  \
0         Learning to speak is always useful. Great!          0   
1  Vocal Exercises begin at 7:50 :   1. Raise arm...          1   
2  Imagine someone being late and walking in at 8:49          0   
3  4:01 "Tempered with love, honesty is a great t...          0   
4  "If you wish people with love its really hard ...          0   
5  i feel like youtube recommendations are person...          1   
6                      I like his head it is shining          0   
7  Very fantastic video.❤❤  I came across this vi...          0   
8  0:13 Intro 0:33 7 deadly sins of speaking 2:45...          0   
9  The trick is, whenever you're talking to peopl...          1   

   BERT_Toxic_Prob  RoBERTa_Pred  RoBERTa_Toxic_Prob  DistilBERT_Pred  \
0         0.008314         

In [10]:
#Testing on youtube comments datasets

import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset

# Step 1: Load CSV file from Kaggle environment
csv_file_path = "/kaggle/input/most-liked-comments-on-youtube/youtube_dataset.csv"  # Replace with your actual CSV file path
comments_df = pd.read_csv(csv_file_path)

# Ensure the column name matches your CSV file (adjust if different)
if "Comment" not in comments_df.columns:
    print("Column 'comment' not found. Available columns:", comments_df.columns)
else:
    print(f"Loaded CSV from {csv_file_path} with {len(comments_df)} comments.")

# Step 2: Convert to Hugging Face Dataset
comments_dataset = Dataset.from_pandas(comments_df[["Comment"]])

# Step 3: Tokenizers for the three fine-tuned models
bert_tokenizer = BertTokenizer.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/models/distilbert_finetuned")

# Step 4: Tokenization function
def tokenize_dataset(dataset, tokenizer, max_length=64):
    def tokenize_function(examples):
        return tokenizer(examples["Comment"], padding="max_length", truncation=True, max_length=max_length)
    return dataset.map(tokenize_function, batched=True)

# Tokenize datasets for all three models
comments_bert = tokenize_dataset(comments_dataset, bert_tokenizer)
comments_roberta = tokenize_dataset(comments_dataset, roberta_tokenizer)
comments_distilbert = tokenize_dataset(comments_dataset, distilbert_tokenizer)

# Step 5: Load the models
bert_model = BertForSequenceClassification.from_pretrained("/kaggle/working/models/bert_finetuned")
roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/working/models/roberta_finetuned")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/models/distilbert_finetuned")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to device
bert_model.to(device)
roberta_model.to(device)
distilbert_model.to(device)

# Step 6: Predict toxicity function
def predict_toxicity(model, tokenizer, dataset, device):
    model.eval()
    predictions = []
    probabilities = []
    comments = dataset["Comment"]
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", padding=True, truncation=True, max_length=64)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = probs.argmax(-1).item()
        toxic_prob = probs[0][1].item()
        predictions.append(pred)
        probabilities.append(toxic_prob)
    return predictions, probabilities

# Step 7: Get predictions for each model
print("Predicting with BERT...")
bert_preds, bert_probs = predict_toxicity(bert_model, bert_tokenizer, comments_bert, device)

print("Predicting with RoBERTa...")
roberta_preds, roberta_probs = predict_toxicity(roberta_model, roberta_tokenizer, comments_roberta, device)

print("Predicting with DistilBERT...")
distilbert_preds, distilbert_probs = predict_toxicity(distilbert_model, distilbert_tokenizer, comments_distilbert, device)

# Step 8: Combine and save results
results_df = pd.DataFrame({
    "Comment": comments_df["Comment"],
    "BERT_Pred": bert_preds,
    "BERT_Toxic_Prob": bert_probs,
    "RoBERTa_Pred": roberta_preds,
    "RoBERTa_Toxic_Prob": roberta_probs,
    "DistilBERT_Pred": distilbert_preds,
    "DistilBERT_Toxic_Prob": distilbert_probs
})

results_csv_path = "/kaggle/working/combined_English_model_predictions_YD.csv"
results_df.to_csv(results_csv_path, index=False)
print(f"Predictions saved to {results_csv_path}")
print(results_df.head(10))  # Show first 10 rows for inspection


Loaded CSV from /kaggle/input/most-liked-comments-on-youtube/youtube_dataset.csv with 19300 comments.


Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Map:   0%|          | 0/19300 [00:00<?, ? examples/s]

Predicting with BERT...
Predicting with RoBERTa...
Predicting with DistilBERT...
Predictions saved to /kaggle/working/combined_English_model_predictions_YD.csv
                                             Comment  BERT_Pred  \
0  The people who liked this comment is officiall...          0   
1           - Wait, it's 7B views\n- Always has been          0   
2  *Teacher: What is the population of the Earth?...          0   
3  Let's be honest this wasn't your recommendatio...          0   
4  Types Of People:\n10% Enjoying Song \n90% Chec...          0   
5  3.2 Million comments if you find mine your a l...          0   
6  claim your “here before 7 billion” tickets her...          1   
7  The ones who are NOT from Tik-Tok can like thi...          0   
8   Song: spanish\nComments: English\nHotel: trivago          0   
9                                    Kimler burda😂🥰🌹          0   

   BERT_Toxic_Prob  RoBERTa_Pred  RoBERTa_Toxic_Prob  DistilBERT_Pred  \
0         0.012601           